# Aula 2 - Solução dos exercícios
Leandro Carísio Fernandes

<br>

Enunciado do exercício da semana:

- Reranqueamento usando um modelo estilo-BERT com o treinamento no dataset do MS MARCO e avaliação no TREC-DL 2020

- O treinamento é igual ao de um classificador binário, que será feito por vocês.

- O que muda é a forma de avaliação: reranqueadores precisam ser alimentados com documentos candidatos (ex: trazidos pelo BM25 - exercício aula 1)

- Sugestão: usar este dataset reduzido do MS MARCO como treinamento, com 10k triplas (query, passagem relevante, passagem não-relevante):
https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv

- Sugestão: usar miniLM (modelo BERT pequeno, 5x mais rapido) para começar o finetuning: https://huggingface.co/nreimers/MiniLM-L6-H384-uncased pois oferece um bom compromisso entre qualidade e velocidade.

- Sugestão: usar este notebook como base: 
[Análise de sentimentos (dataset IMDB) usando um modelo estilo BERT](https://colab.research.google.com/drive/10etP7Lb915EC-uEuf1IKC8DYkyg_om6-?usp=sharing)

- Sugestão de debug: usar este minilm para ver se consegue ndcg ~0.70: https://huggingface.co/cross-encoder/ms-marco-MiniLM-L-6-v2

- Sugestão: fazer overfit em um batch: treinar por 200 epocas um unico batch, e ver se consegue loss=0, e accuracia=100%, ou ndcg=1



<br>


## Instalação da biblioteca, configurações inicias e treinamento/criação do modelo

A célula abaixo contém as variáveis que podem ser configuradas nesse notebook:

In [ ]:
# Configura se é pra treinar o modelo novamente. Se não for, ele vai carregar do google drive
treinar_e_salvar_modelo = False

# Número de épocas usadas no treinamento
epochs = 5

# Seed usada
seed = 123

# É pra fazer o teste usando a implementação do BM25 do pyserin?
executar_implementacao_bm5_pyserini = True

# É pra fazer o teste usando a implementação do BM25 feita na Aula 1?
executar_implementacao_bm5_propria = True

# Na entrada do Dataset, é pra configurar os inputs usando tokenizer.encode_plus(query, passagem, ...) ou 
# usando tokenizer(f'{query} [SEP] {passagem}')?
usar_encode_plus = True

# Total de documentos que vão sair do BM25 pra fazer o reranking
n_doc_reranking = 1000

# Local onde é pra salvar/recuperar o modelo depois do finetuning
local_modelo_tunado = '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula2-classificador-binario-mini-bert/'

# Tamanho do conjunto de treinamento (em porcentagem)
porc_conjunto_treinamento = 0.8

# Nome do modelo pré-treinado que será usado no finetuning
model_name = 'microsoft/MiniLM-L12-H384-uncased'

# Batch size (treinamento)
batch_size = 32

# Tamanho máximo da sequência usada para entrada na rede (não tem memória suficiente pra testar com 512)
max_seq_length = 256

Monta o drive, instala libraries e seta a seed:

In [ ]:
%%time
# Já monta o drive, pois vamos usar o índice invertido da Aula 1 para usar o BM25 implementado também na aula 1
# Além disso, é necessário para salvar/recuperar o modelo tunado
from google.colab import drive
drive.mount('/content/drive')

# Instala libs
!pip install transformers -q
!pip install pyserini
!pip install faiss-cpu

# Importa o PyTorch, random e numpy e seta algum valor na seed deles pra garantir reproducibilidade
import random
import torch
import torch.nn.functional as F
import numpy as np

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 88.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 KB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 5.4 MB/s eta 0:00

Carrega o arquivo msmarco_triples.train.tiny.tsv e retorna a lista de queries com uma passagem e uma classificação (True ou False) indicando se a passagem é ou não relevante para a query:

In [ ]:
%%time
# Lê o arquivo de triplas de treinamento. Só é necessário se for regerar o modelo. Mas considerando que isso é muito rápido, 
# podemos sempre ler esse arquivo. Ainda mais que pode ser útil em casos de testes
from pathlib import Path

# Faz o download das triplas de treinamento
if not Path('msmarco_triples.train.tiny.tsv').is_file():
  !wget https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv # type: ignore

# Esse método pega o conteúdo de msmarco_triples.train.tiny.tsv e 
# gera 3 vetores: Um vetor com as queries, um vetor com as passagens e outro vetor
# com a classificação
def carrega_triplas_treinamento():
  linhas = []
  with open('msmarco_triples.train.tiny.tsv', 'r', encoding='utf-8') as arquivo:
    linhas = [ linha.split('\t') for linha in arquivo]
  queries, passage_relevantes, passage_n_relevantes = zip(*linhas)

  todas_queries = queries * 2
  passage = passage_relevantes + passage_n_relevantes
  classificacao = [True]*len(passage_relevantes) + [False]*len(passage_n_relevantes)

  return todas_queries, passage, classificacao

--2023-03-14 21:49:47--  https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 142.250.99.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8076179 (7.7M) [text/tab-separated-values]
Saving to: ‘msmarco_triples.train.tiny.tsv’

msmarco_triples.tra 100%[===================>]   7.70M  --.-KB/s    in 0.08s   

2023-03-14 21:49:47 (93.7 MB/s) - ‘msmarco_triples.train.tiny.tsv’ saved [8076179/8076179]

CPU times: user 6.51 ms, sys: 6.51 ms, total: 13 ms
Wall time: 213 ms


Faz um shuffle dos dados e separa em conjunto de treinamento e validação. Em cada conjunto, calcula a quantidade de passagens relevantes usadas:

In [ ]:
%%time
# Recupera as queries, passage e classificação (se relevante ou não relevante)...
todas_queries, passage, classificacao = carrega_triplas_treinamento()

# ...faz um shuflle dos dados...
tudo_combinado = list(zip(todas_queries, passage, classificacao))
random.shuffle(tudo_combinado)
todas_queries, passage, classificacao = zip(*tudo_combinado)

# ... e agora divide em conjunto de treinamento e validação
tam_conjunto_treinamento = int(porc_conjunto_treinamento * len(todas_queries))

queries_train = todas_queries[:tam_conjunto_treinamento]
passage_train = passage[:tam_conjunto_treinamento]
classificacao_train = classificacao[:tam_conjunto_treinamento]

queries_val = todas_queries[tam_conjunto_treinamento:]
passage_val = passage[tam_conjunto_treinamento:]
classificacao_val = classificacao[tam_conjunto_treinamento:]

print('Treinamento:')
print(f'total elementos: {len(queries_train)}')
print(f'total relevantes: {sum(classificacao_train)} ({sum(classificacao_train)/len(queries_train)})\n')

print('Validação:')
print(f'total elementos: {len(queries_val)}')
print(f'total relevantes: {sum(classificacao_val)} ({sum(classificacao_val)/len(queries_val)})')

Treinamento:
total elementos: 17600
total relevantes: 8790 (0.4994318181818182)

Validação:
total elementos: 4400
total relevantes: 2210 (0.5022727272727273)
CPU times: user 100 ms, sys: 12.8 ms, total: 113 ms
Wall time: 112 ms


Funções para carregar os dados em estruturas do PyTorch, carregar o modelo pré-treinado e fazer o finetunning:

In [ ]:
%%time
# Aqui são funções que carrega um modelo miniLM e faz o finetuning. Só é executado se for chamado
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from statistics import mean
from torch import nn
from torch import optim
from tqdm.auto import tqdm
from transformers import get_linear_schedule_with_warmup, AutoModelForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from transformers import BatchEncoding

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from torch.utils import data

# Definição para uma classe de Dataset. Basicamente é um getItem que retorna um
# batch_encoding
class Dataset(data.Dataset):
  def __init__(self, tokenizer, queries, passages, labels, max_seq_length = 256):
    self.max_seq_length = max_seq_length
    self.tokenizer = tokenizer
    self.queries = queries
    self.passages = passages
    self.labels = labels
    self.query_passages = [ f"{queries[i]} [SEP] {passages[i]}" for i in range(len(queries)) ]

  def __len__(self):
    return len(self.queries)
    
  def __getitem__(self, idx):
    if usar_encode_plus:
      batch_encoding = self.tokenizer.encode_plus(self.queries[idx], self.passages[idx], add_special_tokens=True,
                                             truncation=True, max_length=self.max_seq_length, padding='max_length', return_attention_mask=True, return_tensors='pt')
      return {
        # Sem o squeeze ele tá gerando um tamanho [1, mas_seq_length], o que dá erro depois na hora de fazer model(**batch.to(device)).
        # Precisamos converter para [mas_seq_length] apenas
        'input_ids': batch_encoding['input_ids'].squeeze(0),
        'attention_mask': batch_encoding['attention_mask'],
        'labels': int(self.labels[idx]),
      }
    else:
      batch_encoding = self.tokenizer(self.query_passages[idx], truncation=True)
      return {
        'input_ids': batch_encoding['input_ids'],
        'attention_mask': batch_encoding['attention_mask'],
        'labels': int(self.labels[idx]),
      }

# Finetuning
# We first define the evaluation function to measure accuracy and loss
def evaluate(model, dataloader, set_name):
  print(f'****************** Avaliando: {set_name} **********************')
  losses = []
  correct = 0
  model.eval()
  with torch.no_grad():
    for batch in tqdm(dataloader, mininterval=0.5, desc=set_name, disable=False):
      outputs = model(**batch.to(device))
      loss_val = outputs.loss
      losses.append(loss_val.cpu().item())
      preds = outputs.logits.argmax(dim=1)
      correct += (preds == batch['labels']).sum().item()

  print(f'{set_name} loss: {mean(losses):0.3f}; {set_name} accuracy: {correct / len(dataloader.dataset):0.3f}')
  print(f'***************************************************************')

def treina_modelo(model, optimizer, dataloader_train, dataloader_val, epochs = 5):
  num_training_steps = epochs * len(dataloader_train)
  # Warm up is important to stabilize training.
  num_warmup_steps = int(num_training_steps * 0.1)
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)

  # First validation to check if evaluation code is working and accuracy is random as expected 
  evaluate(model=model, dataloader=dataloader_val, set_name='Validation')

  # Training loop
  for epoch in tqdm(range(epochs), desc='Epochs'):
    model.train()
    train_losses = []
    for batch in tqdm(dataloader_train, mininterval=0.5, desc='Train', disable=False):
      optimizer.zero_grad()
      outputs = model(**batch.to(device))
      loss = outputs.loss
      loss.backward()
      optimizer.step()
      scheduler.step()
      train_losses.append(loss.cpu().item())

    print(f'Epoch: {epoch + 1} Training loss: {mean(train_losses):0.2f}')
    evaluate(model=model, dataloader=dataloader_val, set_name='Validation')

def salvar_modelo_e_tokenizer_no_drive(model, tokenizer):
  model.save_pretrained(local_modelo_tunado)
  tokenizer.save_pretrained(local_modelo_tunado)
  print('Modelo salvo')

def criar_datasets(tokenizer):
  dataset_train = Dataset(tokenizer, queries_train, passage_train, classificacao_train, max_seq_length)
  dataset_val = Dataset(tokenizer, queries_val, passage_val, classificacao_val, max_seq_length)
  
  return dataset_train, dataset_val

def criar_dataloaders(tokenizer, dataset_train, dataset_val):
  collate_fn = lambda batch: BatchEncoding(tokenizer.pad(batch, return_tensors='pt'))
  dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
  dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

  return dataloader_train, dataloader_val

def carregar_modelo_miniLM_e_fazer_finetuning(epochs = 5):
  # Carrega o tokenizador
  tokenizer = AutoTokenizer.from_pretrained(model_name)

  # Cria os datasets
  dataset_train, dataset_val = criar_datasets(tokenizer)

  # Cria os dataloaders de treinamento e validação
  dataloader_train, dataloader_val = criar_dataloaders(tokenizer, dataset_train, dataset_val)

  # Cria o modelo
  model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
  print('Parameters', model.num_parameters())

  # Cria um otimizador Adam
  optimizer = AdamW(model.parameters(), lr=5e-5)
  treina_modelo(model, optimizer, dataloader_train, dataloader_val, epochs)

  # Salva o modelo
  salvar_modelo_e_tokenizer_no_drive(model, tokenizer)

  # Retorna
  return model, tokenizer

CPU times: user 2.35 s, sys: 1.15 s, total: 3.5 s
Wall time: 3.22 s


Função para carregar o modelo a partir do drive:

In [ ]:
%%time
# Função para carregar um modelo do drive
def carregar_modelo_e_tokenizer_do_drive():
  tokenizer = AutoTokenizer.from_pretrained(local_modelo_tunado)
  model = AutoModelForSequenceClassification.from_pretrained(local_modelo_tunado).to(device)
  return model, tokenizer

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 7.87 µs


Carrega o modelo e tokenizador:

In [ ]:
# Aqui é que carregamos ou criamos o modelo
if treinar_e_salvar_modelo:
  model, tokenizer = carregar_modelo_miniLM_e_fazer_finetuning(epochs)
else:
  model, tokenizer = carregar_modelo_e_tokenizer_do_drive()

## Reranking com a implementação do índice invertido e BM25 feita na Aula 1

### Implementação

Faz a implementação do índice invertido (o código veio do caderno da aula anterior):

In [ ]:
%%time
from pyserini.analysis import Analyzer, get_lucene_analyzer
from collections import Counter
import array
import pickle

# Definição de uma classe para índice invertido
class IndiceInvertido:

  # Recebe 'tokenizar', uma função tokenizadora
  def __init__(self, tokenizar = None):
    # Cria um índice invertido vazio
    self.indice = {}
    # Cria um índice de tamanho de documentos vazio
    self.tamanho_doc = {}
    # Guarda o total de documentos adicionados
    self.n_docs = 0

    # Se não passou uma função tokenizadora como parâmetro, considera o do Lucene som Porter stemmer:
    if (tokenizar is None):
      tokenizar = Analyzer(get_lucene_analyzer(stemmer='porter')).analyze
    self.tokenizar = tokenizar

  def adiciona_doc(self, id_doc, conteudo_doc=None, tokens=None):
    if (tokens is None):
      tokens = self.tokenizar(conteudo_doc)

    contador_tokens_do_documento = Counter(tokens)
    for token, n_ocorrencias in contador_tokens_do_documento.items():
      # Tenta converter o id do documento para int (ocupa menos memória que uma string)
      # Usa um array de 4bytes para e um de 2bytes para o número de ocorrências
      int_id_doc = int(id_doc)
      self.indice.setdefault(token, {"id_doc": array.array("L", []), "n_ocorrencias": array.array("L", [])})['id_doc'].append(int_id_doc)
      self.indice.setdefault(token, {"id_doc": array.array("L", []), "n_ocorrencias": array.array("L", [])})['n_ocorrencias'].append(n_ocorrencias)
    
    self.n_docs += 1
    self.tamanho_doc[int_id_doc] = len(tokens)

def copiar_indice_invertido_do_drive_em_iidx_ms_marco():
  # Demorando cerca de 1min pra copiar as informações do drive e abrir aqui
  print('Copiando arquivo de índice do drive...')
  !cp '/content/drive/My Drive/IA368-DD_deep_learning_busca/iidx_ms_marco.pickle' './iidx_ms_marco.pickle'  # type: ignore
  print('Copiando arquivo de tamanho do documento do drive...')
  !cp '/content/drive/My Drive/IA368-DD_deep_learning_busca/tamanho_doc_ms_marco.pickle' './tamanho_doc_ms_marco.pickle'  # type: ignore

  # Remonta o índice invertido
  print('Lendo arquivo de índice...')
  with open('iidx_ms_marco.pickle', 'rb') as f:
    iidx_ms_marco.indice = pickle.load(f)
  # Remonta o índice de tamanho de documento
  print('Lendo arquivo de tamanho do documento...')
  with open('tamanho_doc_ms_marco.pickle', 'rb') as f:
    iidx_ms_marco.tamanho_doc = pickle.load(f)
  # Recalcula o total de documentos
  iidx_ms_marco.n_docs = len(iidx_ms_marco.tamanho_doc)

# Variável que guarda o índice invertido
iidx_ms_marco = IndiceInvertido()
copiar_indice_invertido_do_drive_em_iidx_ms_marco()

Copiando arquivo de índice do drive...
Copiando arquivo de tamanho do documento do drive...
Lendo arquivo de índice...
Lendo arquivo de tamanho do documento...
CPU times: user 5.32 s, sys: 6.34 s, total: 11.7 s
Wall time: 52.3 s


Traz o buscador BM25 da aula 1 pra cá (na verdade o buscador aqui pode fazer o cálculo tanto com BM25 quanto com TF-IDF):

In [ ]:
%%time
# Buscador com BM25

import math

class PesquisaComTFIDF:

  def __init__(self, indiceInvertido=IndiceInvertido(), k1 = 0.9, b = 0.4, adicionar_ao_idf = 0):
    self.indiceInvertido = indiceInvertido
    self.adicionar_ao_idf = adicionar_ao_idf
    self.calcula_tam_medio_doc_no_indice()
    self.k1 = k1
    self.b = b
    self.precalcula_idf()
  
  def calcula_tam_medio_doc_no_indice(self):
    self.avgdl = sum(self.indiceInvertido.tamanho_doc.values()) / self.indiceInvertido.n_docs

  def precalcula_idf(self):
    # Número de documento do corpus está presente no objeto indiceInvertido
    N = self.indiceInvertido.n_docs
    # Varre todos os tokens do índice. Os tokens são as chaves do indiceInvertido.indice
    for token in self.indiceInvertido.indice.keys():
      # O número de documentos que possui o token é calculado pelo tamanho da lista de id_doc:
      n_doc_token = len(self.indiceInvertido.indice[token]['id_doc'])
      # Isso já é o suficiente pra calcular o idf
      idf_token = math.log( ((self.indiceInvertido.n_docs - n_doc_token + 0.5)/(n_doc_token + 0.5)) + self.adicionar_ao_idf )
      # E agora, vamos colocar essa informação no índice
      self.indiceInvertido.indice[token]['idf'] = idf_token

  def calcula_tf_idf_para_um_token_e_salva(self, token):
    # O cálculo do tf_idf exibe apenas multiplicar o idf do termo (que já temos calculado) pelo tf(termo, documento)
    # A fórmula do tf = a frequência do termo no documento dividido pelo total de termos do documento
    # Logo, tf_idf = idf * frequencia_token_no_doc / tamanho_doc
    #             - o idf do token já está precalculado e consta do índice: self.indiceInvertido.indice[token]['idf']
    #             - o tamanho_doc também já consta do índice: self.indiceInvertido.tamanho_doc[id_doc] (para certo id_doc)
    #             - a frequência do token no documento também consta do índice: self.indiceInvertido.indice[token]['n_ocorrencias']
    # Juntando tudo:
    # Recupera o idf
    idf_token = self.indiceInvertido.indice[token]['idf']
    # Calcula o tf_idf
    zip_id_freq = zip(self.indiceInvertido.indice[token]['id_doc'], self.indiceInvertido.indice[token]['n_ocorrencias'])
    #tf_idf = { id_doc: idf_token * freq_token_no_doc / self.indiceInvertido.tamanho_doc[id_doc] for (id_doc, freq_token_no_doc) in zip_id_freq ]}
    tf_idf = array.array('f', [idf_token * freq_token_no_doc / self.indiceInvertido.tamanho_doc[id_doc] for (id_doc, freq_token_no_doc) in zip_id_freq ])
    # Salva o tf_idf no índice
    self.indiceInvertido.indice[token]['tf_idf'] = tf_idf

  def calcula_bm25_para_um_token_e_salva(self, token):
    # O cálculo do BM25 para determinada query é a multiplicação do idf pela frequência do termo no documento * (k1 + 1)
    # Além disso, é dividido pela frequencia do termo no documento + k1 * (1 - b + b * tamanho_doc/avgdl)
    # Para não ter que ficar fazendo várias multiplicações/divisões, vamos resolver o que dá pra resolver antes.
    # Sobre isso, o denominador fica: k1 - k1*b + k1*b*tamanho_doc/avgdl
    # Então precisamos de três variáveis:
    #         idf_x_k1_mais_1 = idf*(k1 + 1)
    #         k1_menos_k1_x_b = k1 - k1*b
    #         k1_x_b_d_avgdl = k1*b/avgdl
    idf_x_k1_mais_1 = self.indiceInvertido.indice[token]['idf']*(self.k1 + 1)
    k1_menos_k1_x_b = self.k1 - self.k1*self.b
    k1_x_b_d_avgdl = self.k1 * self.b / self.avgdl

    # Juntando tudo, podemos calcular o score pelo BM25
    zip_id_freq = zip(self.indiceInvertido.indice[token]['id_doc'], self.indiceInvertido.indice[token]['n_ocorrencias'])
    #bm25 = { id_doc: idf_x_k1_mais_1 * freq_token_no_doc / (freq_token_no_doc + k1_menos_k1_x_b + k1_x_b_d_avgdl*self.indiceInvertido.tamanho_doc[id_doc]) for (id_doc, freq_token_no_doc) in zip_id_freq }
    bm25 = array.array('f', [ idf_x_k1_mais_1 * freq_token_no_doc / (freq_token_no_doc + k1_menos_k1_x_b + k1_x_b_d_avgdl*self.indiceInvertido.tamanho_doc[id_doc]) for (id_doc, freq_token_no_doc) in zip_id_freq ])
    # Salva o bm25 no índice
    self.indiceInvertido.indice[token]['bm25'] = bm25

  def tokenizar(self, query):
    return self.indiceInvertido.tokenizar(query)

  def pesquisar(self, query, metodo = 'tf_idf'):
    # Tokeniza a query
    tokens = self.tokenizar(query)

    # Se não tem token para ser pesquisado, retorna conjunto vazio
    if (len(tokens) == 0):
      return []

    # Guarda um dicionário onde a chave é o id do documento e o valor é o score desse documento para a query pesquisada
    docs_retornado_com_score = Counter({})

    # Função para o cálculo de score
    funcao_calc_salvar_scores_docs_deste_token = self.calcula_tf_idf_para_um_token_e_salva if metodo == 'tf_idf' else self.calcula_bm25_para_um_token_e_salva

    # Faz a pesquisa de documentos. Para isso iteramos todos os tokens da query
    for token in tokens:
      # É possível que a query contenha algum termo que não foi indexado. Se isso ocorrer,
      # entende-se que a frequência desse token em qualquer documento é 0, já que não pode ser encontrado
      if token not in self.indiceInvertido.indice:
        continue

      # Pega a lista de documentos que será analisado
      docs_que_tem_token = self.indiceInvertido.indice[token]['id_doc']
      
      # Se for a primeira vez que esse token é pesquisado, é necessário calcular o tf_idf relacionado
      # a ele e salvar. Se já tiver sido feito antes, já podemos buscar o cálculo pronto (que funciona
      # como um cache. Isso é útil no caso de várias pesquisas seguidas)
      if metodo not in self.indiceInvertido.indice[token].keys():
        funcao_calc_salvar_scores_docs_deste_token(token)
      score_dos_docs_deste_token = self.indiceInvertido.indice[token][metodo]

      # Agora já temos calculado o tf_idf de todos os documentos desse token. Só adiciona ao acumulador de score atual
      # docs_retornado_com_score += score_dos_docs_deste_token -> Se fosse usar dict direto no índice seria assim, mas a memória não está aguentando guardar os scores de ambos
      for id_doc, score_par_doc_token in zip(docs_que_tem_token, score_dos_docs_deste_token):
        docs_retornado_com_score[id_doc] += score_par_doc_token

    # Agora converte esse dict em uma lista de tuplas com a chave (id_doc) e valor (score_do_doc)
    docs_com_score = list(docs_retornado_com_score.items())

    # E ordena do mais relevante para o menos relevante
    return sorted(docs_com_score, key=lambda x: x[1], reverse=True)

CPU times: user 652 µs, sys: 0 ns, total: 652 µs
Wall time: 430 µs


Lê os dados da coleção MS MARCO e guarda numa lista o conteúdo dos documentos (passagens). Isso é necessário pois no reranking precisamos acessar o texto dos documentos.

In [ ]:
%%time
# Carrega e descompacta o arquivo de msmarco. Será necessário buscar o texto dos documentos
# depois do BM25
from pathlib import Path
import hashlib

str_arquivo = './collections/msmarco-passage/collectionandqueries.tar.gz'
md5_esperado = '31644046b18952c1386cd4564ba2ae69'

if not Path(str_arquivo).is_file():
  !mkdir collections/msmarco-passage # type: ignore
  !wget https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz -P collections/msmarco-passage # type: ignore
  !tar xvfz collections/msmarco-passage/collectionandqueries.tar.gz -C collections/msmarco-passage # type: ignore
else:
  print('Arquivo já foi baixado, não precisa fazer o download novamente')

with open(str_arquivo, 'rb') as arquivo:
  data = arquivo.read()
  md5_arquivo = hashlib.md5(data).hexdigest()
  del data
  print('MD5 bate com o esperado' if md5_esperado == md5_arquivo else 'MD5 é diferente do esperado, é necessário fazer o download novamente.')


# Considera que o arquivo collection.tsv está ordenado e cria uma lista contendo todos os documentos.
# O índice i da lista contém o texto do documento de mesmo id
texto_docs = []
with open('collections/msmarco-passage/collection.tsv', encoding='utf-8') as f:
  for idx, line in enumerate(f):
    doc_id, contents = line.rstrip().split('\t')
    texto_docs.append(contents)
    if idx % 1000000 == 0:
      print(f"Carregado {idx} docs...")

mkdir: cannot create directory ‘collections/msmarco-passage’: No such file or directory
--2023-03-14 21:50:56--  https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1057717952 (1009M) [application/gzip]
Saving to: ‘collections/msmarco-passage/collectionandqueries.tar.gz’

collectionandquerie 100%[===================>]   1009M  27.8MB/s    in 33s     

2023-03-14 21:51:29 (30.6 MB/s) - ‘collections/msmarco-passage/collectionandqueries.tar.gz’ saved [1057717952/1057717952]

collection.tsv
qrels.dev.small.tsv
qrels.train.tsv
queries.dev.small.tsv
queries.dev.tsv
queries.eval.small.tsv
queries.eval.tsv
queries.train.tsv
MD5 bate com o esperado
Carregado 0 docs...
Carregado 1000000 docs...
Carregado 2000000 docs...
Ca

Carrega os tópicos TREC DL 2020 e cria uma função para rerankear documentos

In [ ]:
%%time
from pyserini.search import get_topics

# Carrega a lista de queries do TREC DL 2020
topics = get_topics('dl20')

# query_id pode ser qualquer coisa, eu só uso ele pra imprimir do lado da query pra facilitar debug
def reranking_docs(query_id, query, doc_ids):
  model.eval()
  
  passagens = [ texto_docs[id] for id in doc_ids ]
  dataset_teste = Dataset(tokenizer, [query]*len(doc_ids), passagens, [0.5]*len(doc_ids), max_seq_length)
  
  collate_fn = lambda batch: BatchEncoding(tokenizer.pad(batch, return_tensors='pt'))
  dataloader_teste = DataLoader(dataset_teste, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

  prob_classe_1 = []
  #return dataset_teste
  with torch.no_grad():
    for batch in tqdm(dataloader_teste, mininterval=0.5, desc=f'Query {query_id}: {query}', disable=False):
      outputs = model(**batch.to(device))
      prob = torch.nn.functional.softmax(outputs.logits, dim=1)
      classe_0, classe_1 = zip(*prob)
      prob_classe_1.extend( list(classe_1) )

  docs_com_score = list(zip(doc_ids, prob_classe_1))

  torch.cuda.empty_cache()

  return sorted(docs_com_score, key=lambda x: x[1], reverse=True)

CPU times: user 2.28 s, sys: 184 ms, total: 2.46 s
Wall time: 2.7 s


Cria um buscador BM25 e traz os primeiros 1.000 resultados. Aplica o modelo de reranking nesses resultados e confere os 5 primeiros e últimos resultados:

In [ ]:
%%time

# Cria um buscador
buscador_bm25 = PesquisaComTFIDF(iidx_ms_marco, 1)

# Teste de uma query qualquer, pra ver se o resultado pelo menos faz algum sentido:
query_id = 735922 #23849
query = topics[735922]['title']
indices, _ = zip(*buscador_bm25.pesquisar(query, 'bm25'))

# Agora vamos adicionar um documento fake que não tem nada a ver com a query. Vamos usar esse documento pra rankear também, pra ver se ele vai lá pro final da lista
texto_docs.append("feijão tropeiro is one of the best Brazilian foods. there isn't a single soul that doesn't like it")
idx_fake_doc = len(texto_docs) - 1
docs_score = reranking_docs(735922, query, (idx_fake_doc, *indices[:n_doc_reranking-1]))

len(f'Total de documentos: {docs_score}')

print('Alto no ranking...')
for id_doc, score in docs_score[0:5]:
  print(f'{id_doc}, {score}: {texto_docs[int(id_doc)]}')

print('\nBaixo no ranking...')
for id_doc, score in docs_score[-5:]:
  print(f'{id_doc}, {score}: {texto_docs[int(id_doc)]}')

# Agora temos que excluir o documento da lista de textos:
print(f'\nRetirando o texto "{texto_docs.pop()}"...\n')

# Vamos ver em que posição do ranking esse documento ficou:
for i, item in enumerate(docs_score):
    if item[0] == idx_fake_doc:
        print(f'\nO texto fake ficou na posição {i} com score {item[1]}')
        break

Query 735922: what is crimp oil:   0%|          | 0/32 [00:00<?, ?it/s]

Alto no ranking...
8626892, 0.997641921043396: Crimp Oil is a 100% natural blend of essential oils and plant extracts that aids in the recovery of climbing-related injuries.
8626887, 0.9969774484634399: Metolius Crimp Oil. 1  A healing massage oil for climbers hands and muscles. 2  Reduces pain and swelling in tendons, joints and muscles. 3  100% natural blend of essential oils and plant extracts. 4  Designed by climbers. 5  Available in 10 and 30 ml bottles.
7307866, 0.9961373209953308: crimp (plural crimps) 1  A fastener or a fastening method that secures parts by bending metal around a joint and squeezing it together, often with a tool that adds indentations to capture the parts. ( 2 obsolete, Britain, dialect) A coal broker. ( 3 obsolete) One who decoys or entraps men into the military or naval service.
8765058, 0.9916137456893921: Definition of crimp. 1 : something produced by or as if by crimping: such asa : a section of hair artificially waved or curledb : a succession of waves 

### Resultado

Executa todas as queries no buscador BM25 implementado:

In [ ]:
%%time
from pyserini.search import get_qrels

# Carrega a lista de queries do TREC DL 2020
qrels_id = list(get_qrels('dl20-passage').keys())

# Roda todas as queries
def run_all_queries_bm25_e_rerank(file, topics, buscador_bm25):
  with open(file, 'w') as runfile:
    cnt = 0
    print('Running {} queries in total'.format(len(topics)))
    for id in topics:
      if cnt % 10 == 0:
        print(f'{cnt} queries completed')
      cnt += 1

      # Pega a query
      query = topics[id]['title']

      if id not in qrels_id:
        print(f'Query {id} não está no qrels. Pulando: {query}')
        continue

      # Usa o BM25 para buscar
      indices, score = zip(*buscador_bm25.pesquisar(query, 'bm25'))
      # Agora faz o reranking
      docs_score = reranking_docs(id, query, indices[:n_doc_reranking])

      for i in range(0, min(len(docs_score), n_doc_reranking)): # Pega os primeiros 1000 resultados
        _ = runfile.write('{} Q0 {} {} {:.6f} BM_25_RERANKING_MINILM\n'.format(id, docs_score[i][0], i+1, docs_score[i][1]))

if executar_implementacao_bm5_propria:
  print('Rodando pesquisa com BM25 e reranking usando miniLM')
  run_all_queries_bm25_e_rerank('run-pesquisa-bm25-reranking.dl20.txt', topics, buscador_bm25)
  !python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl20-passage run-pesquisa-bm25-reranking.dl20.txt #type: ignore

Rodando pesquisa com BM25 e reranking usando miniLM
Running 200 queries in total
0 queries completed
Query 735922 não está no qrels. Pulando: what is crimp oil


Query 23849: are naturalization records public information:   0%|          | 0/32 [00:00<?, ?it/s]

Query 514096 não está no qrels. Pulando: the after hours clinic


Query 156498: do google docs auto save:   0%|          | 0/32 [00:00<?, ?it/s]

Query 258062: how long does it take to remove wisdom tooth:   0%|          | 0/32 [00:00<?, ?it/s]

...

Query 1103153 não está no qrels. Pulando: who is thomas m cooley
Query 1129081 não está no qrels. Pulando: in healthcare what does iihi stand for
190 queries completed
Query 3505 não está no qrels. Pulando: how do they do open heart surgery
Query 444389 não está no qrels. Pulando: magnesium definition chemistry
Query 1114286 não está no qrels. Pulando: what is in the meat group
Query 1045109 não está no qrels. Pulando: who owns barnhart crane
Query 1114166 não está no qrels. Pulando: what is it called when your blood becomes too thin


Query 673670: what is a alm:   0%|          | 0/32 [00:00<?, ?it/s]

Query 1108729 não está no qrels. Pulando: what temperature and humidity to dry sausage
Query 245052 não está no qrels. Pulando: how long do financial institutions keep records after closing


Query 324585: how much money do motivational speakers make:   0%|          | 0/32 [00:00<?, ?it/s]

Query 543273 não está no qrels. Pulando: weather in antigua november
2023-03-12 21:44:46.362532: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-12 21:44:46.362664: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-12 21:44:46.362682: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
jtreceval-0.0.5-jar-with-dependencies.jar: 1.79MB [00:00, 5.07MB/s]                
Running comman

## Reranking com a implementação do BM25 do Pyserini

### Carrega o LuceneSearcher

In [ ]:
%%time
from pyserini.search.lucene import LuceneSearcher
import json
from pyserini.search import get_qrels

if executar_implementacao_bm5_pyserini:
  # Copia o índice do pyserini que já foi gerado na Aula 1 e está no google drive.
  !cp '/content/drive/My Drive/IA368-DD_deep_learning_busca/indexes.tar.gz' './indexes.tar.gz'  # type: ignore
  !tar xvfz indexes.tar.gz  # type: ignore

  searcher = LuceneSearcher('indexes/lucene-index-msmarco-passage')
  searcher.set_bm25(k1=float(0.9), b=float(0.4))


indexes/
indexes/lucene-index-msmarco-passage/
indexes/lucene-index-msmarco-passage/_0_Lucene90_0.doc
indexes/lucene-index-msmarco-passage/_0_Lucene90_0.dvm
indexes/lucene-index-msmarco-passage/_8.fnm
indexes/lucene-index-msmarco-passage/_0.fnm
indexes/lucene-index-msmarco-passage/_4.tvx
indexes/lucene-index-msmarco-passage/_0_Lucene90_0.pos
indexes/lucene-index-msmarco-passage/_5_Lucene90_0.tip
indexes/lucene-index-msmarco-passage/_2.tvd
indexes/lucene-index-msmarco-passage/_4.fdx
indexes/lucene-index-msmarco-passage/_0.nvd
indexes/lucene-index-msmarco-passage/_3.tvx
indexes/lucene-index-msmarco-passage/_8.fdx
indexes/lucene-index-msmarco-passage/_4_Lucene90_0.dvd
indexes/lucene-index-msmarco-passage/_4.fdt
indexes/lucene-index-msmarco-passage/_6.tvm
indexes/lucene-index-msmarco-passage/_7.nvd
indexes/lucene-index-msmarco-passage/_2.fnm
indexes/lucene-index-msmarco-passage/_2.nvd
indexes/lucene-index-msmarco-passage/_6_Lucene90_0.dvd
indexes/lucene-index-msmarco-passage/_1_Lucene90_0.

### Resultado

Executa todas as queries usando o LuceneSearcher:

In [ ]:
%%time
def run_all_queries_bm25_pyserini_e_rerank(file, topics, searcher):
  with open(file, 'w') as runfile:
    cnt = 0
    print('Running {} queries in total'.format(len(topics)))
    for id in topics:
      if cnt % 10 == 0:
        print(f'{cnt} queries completed')
      cnt += 1

      # Pega a query
      query = topics[id]['title']

      if id not in qrels_id:
        print(f'Query {id} não está no qrels. Pulando: {query}')
        continue

      # Usa o BM25 para buscar
      hits = searcher.search(query, n_doc_reranking)
      indices = [ int((json.loads(hit.raw))['id']) for hit in hits ]
      # Agora faz o reranking
      docs_score = reranking_docs(id, query, indices)

      for i in range(0, min(len(docs_score), n_doc_reranking)): # Pega os primeiros 1000 resultados
        _ = runfile.write('{} Q0 {} {} {:.6f} BM_25_RERANKING_MINILM_PYSERINI\n'.format(id, docs_score[i][0], i+1, docs_score[i][1]))


if executar_implementacao_bm5_pyserini:
  print('Rodando pesquisa com BM25 e reranking usando miniLM')
  run_all_queries_bm25_pyserini_e_rerank('run-pesquisa-bm25-reranking-pyserini.dl20.txt', topics, searcher)
  !python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl20-passage run-pesquisa-bm25-reranking-pyserini.dl20.txt #type: ignore

Rodando pesquisa com BM25 e reranking usando miniLM
Running 200 queries in total
0 queries completed
Query 735922 não está no qrels. Pulando: what is crimp oil


Query 23849: are naturalization records public information:   0%|          | 0/32 [00:00<?, ?it/s]

Query 514096 não está no qrels. Pulando: the after hours clinic


Query 156498: do google docs auto save:   0%|          | 0/32 [00:00<?, ?it/s]

Query 258062: how long does it take to remove wisdom tooth:   0%|          | 0/32 [00:00<?, ?it/s]

...

Query 1103153 não está no qrels. Pulando: who is thomas m cooley
Query 1129081 não está no qrels. Pulando: in healthcare what does iihi stand for
190 queries completed
Query 3505 não está no qrels. Pulando: how do they do open heart surgery
Query 444389 não está no qrels. Pulando: magnesium definition chemistry
Query 1114286 não está no qrels. Pulando: what is in the meat group
Query 1045109 não está no qrels. Pulando: who owns barnhart crane
Query 1114166 não está no qrels. Pulando: what is it called when your blood becomes too thin


Query 673670: what is a alm:   0%|          | 0/32 [00:00<?, ?it/s]

Query 1108729 não está no qrels. Pulando: what temperature and humidity to dry sausage
Query 245052 não está no qrels. Pulando: how long do financial institutions keep records after closing


Query 324585: how much money do motivational speakers make:   0%|          | 0/32 [00:00<?, ?it/s]

Query 543273 não está no qrels. Pulando: weather in antigua november
2023-03-12 21:51:56.547066: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-12 21:51:56.547190: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-12 21:51:56.547211: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping down

## Conclusões


Fiz diversos testes usando o modelo pré-treinado 'microsoft/MiniLM-L12-H384-uncased' e fazendo finetuning usando o dataset reduzido do MS MARCO (10.000 registros de passagens relevantes e 10.000 de não relevantes). Em todos os casos foram usadas apenas 5 épocas de treinamento, pois não houve melhora significativa usando mais épocas.

Os resultados foram testados com algumas combinações de % de registro da base usada para treinamento/validação, seeds e se usou tokenizer(f'{query} [SEP] {passagem}') ou tokenizer.encode_plus(query, passagem, ...).

Além disso, foi testado o resultado com a implementação do BM25 do Pyserini e, em alguns casos, com a implementação feita na aula anterior. Na coluna nDCG@10 das tabelas abaixo, o primeiro resultado sempre se refere ao uso do BM25 do Pyserini e, o segundo, da implementação feita. No BM25, foi utilizado k1 = 0.9 e b = 0.4 para manter os mesmos resultados da aula anterior. Nas tabelas, o X indica que o resultado não foi gerado.

<br>

1. Usando tokenizer(f'{query} [SEP] {passagem}'):

<center>

| % de registros usados para treinamento | seed | nDCG@10       |
|----------------------------------------|------|---------------|
| 0.8                                    | 42   | 0.605 / 0.605 |
| 0.8                                    | 123  | 0.566 / 0.566 |
| 0.7                                    | 42   | 0.5849 / X    |
| 0.7                                    | 123  | 0.5919 / X    |

</center>

<br>

2. tokenizer.encode_plus(query, passagem, ...):

<center>

| % de registros usados para treinamento | seed | nDCG@10         |
|----------------------------------------|------|-----------------|
| 0.8                                    | 42   | 0.5852 / X      |
| 0.8                                    | 123  | 0.6198 / 0.6212 |
| 0.7                                    | 42   | 0.5474 / X      |
| 0.7                                    | 123  | 0.5981 / X      |


</center>



In [ ]:
# # Cria um dataset só com passagens relevantes pra testar o resultado do treinamento

# linhas = []
# with open('msmarco_triples.train.tiny.tsv', 'r', encoding='utf-8') as arquivo:
#   linhas = [ linha.split('\t') for linha in arquivo]
# queries, passage_relevantes, passage_n_relevantes = zip(*linhas)
# class_teste = [1]*10
# dataset_teste = Dataset(tokenizer, queries[1:10], passage_relevantes[1:10], class_teste, max_seq_length)

# collate_fn = lambda batch: BatchEncoding(tokenizer.pad(batch, return_tensors='pt'))
# dataloader_teste = DataLoader(dataset_teste, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# prob_classe_1 = []

# with torch.no_grad():
#   for batch in tqdm(dataloader_teste, mininterval=0.5, desc=f'Query {query_id}: {query}', disable=False):
#     outputs = model(**batch.to(device))
#     prob = torch.nn.functional.softmax(outputs.logits, dim=1)
#     classe_0, classe_1 = zip(*prob)
#     prob_classe_1.extend( list(classe_1) )
# # Tudo deve ser próximo de 1
# print(*prob_classe_1, sep='\n')
# print(*class_teste, sep='\n')

